In [100]:
import pandas as pd
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
import matplotlib.pyplot as plt
import seaborn as sns
import os
from preprocessing import Feature_Extractor, data_preprocessing


In [101]:
symbol = "STB"
predictest = pd.read_csv(f'{symbol}_LSTMGRU_evaluate_result.csv')

In [102]:
predictest.dtypes

Unnamed: 0     int64
Date          object
predicted     object
dtype: object

In [103]:
def convert_to_float(x):
    return float(x[0])

In [104]:
if predictest['predicted'].apply(lambda x: isinstance(x, str)).any():
    predictest['predicted'] = predictest['predicted'].apply(eval).apply(convert_to_float)
predictest.dtypes

Unnamed: 0      int64
Date           object
predicted     float64
dtype: object

In [105]:
#Lưu file mới
predictest = predictest.drop(columns=['Unnamed: 0'], errors='ignore')
output_file_path = os.path.join(f'{symbol}_LSTMGRU_evaluate_result.csv')
predictest.to_csv(output_file_path, index=False)

In [106]:
df = pd.read_csv(f'DataTrainLSTM/{symbol}.csv')
df.head


<bound method NDFrame.head of             Date    Close     Open     High      Low
0     2019-01-28  12250.0  12350.0  12350.0  12100.0
1     2019-01-29  12150.0  12200.0  12200.0  12050.0
2     2019-01-30  12000.0  12150.0  12200.0  12000.0
3     2019-01-31  12300.0  12100.0  12400.0  12000.0
4     2019-02-01  12400.0  12300.0  12400.0  12200.0
...          ...      ...      ...      ...      ...
1327  2024-05-27  28700.0  28800.0  28950.0  28350.0
1328  2024-05-28  28850.0  28800.0  29000.0  28600.0
1329  2024-05-29  28250.0  28900.0  28900.0  28250.0
1330  2024-05-30  27850.0  28000.0  28250.0  27600.0
1331  2024-05-31  27850.0  28050.0  28200.0  27800.0

[1332 rows x 5 columns]>

In [107]:
# Hợp nhất hai tệp CSV theo cột Date
df_merged = pd.merge(df, predictest, on='Date', how='left')
df_merged['Close'] = df_merged['predicted'].combine_first(df_merged['Close'])
df_merged.drop(columns=['predicted'], inplace=True)
cutoff_date = '2024-05-17'
df_filtered = df_merged[df_merged['Date'] <= cutoff_date]
output_folder = 'PredictTestData'
output_file_path = os.path.join(output_folder, f'{symbol}LSTMGRUpredictions.csv')
df_filtered.to_csv(output_file_path, index=False)